In [1]:
from dotenv import load_dotenv
from IPython.display import display, Markdown
load_dotenv()

True

## Creating subagents

In [2]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [12]:
from langchain.agents import create_agent
from langchain_aws import ChatBedrock

# 1. CONFIGURACIÓN PARA DEEPSEEK-R1 (Razonamiento Complejo)
# Ideal para agentes que necesitan planificar pasos lógicos.
# llm = ChatBedrock(
#     model_id="us.deepseek.r1-v1:0",  # ID oficial validado
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.6, # DeepSeek recomienda 0.6 para razonamiento
#         "max_tokens": 8192,  # Recomendado para no degradar calidad del CoT
#         "top_p": 0.95,
#     }
# )


# llm = ChatBedrock(
#     model_id="us.deepseek.v3-v1:0", # Prueba este primero
#     region_name="us-east-1",        # O us-west-2
#     model_kwargs={
#         "temperature": 0.7,
#         "max_tokens": 4096
#     }
# )
# from langchain_aws import ChatBedrock
# llm = ChatBedrock(
# model_id="us.meta.llama4-scout-17b-instruct-v1:0",  # Nota el prefijo "us."
# # model_id="cohere.command-r-plus-v1:0",
# region_name="us-east-1",
# model_kwargs={
# "temperature": 0.5,
# "max_tokens": 2048,
# "top_p": 0.9,
# }
# )


# llm = ChatBedrock(
#     model_id="us.meta.llama4-maverick-17b-instruct-v1:0",  # Nota el prefijo "us."
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.5,
#         "max_tokens": 2048,
#         "top_p": 0.9,
#     }
# )

llm = ChatBedrock(
    model_id="amazon.nova-lite-v1:0",  # Nota el prefijo "us."
    region_name="us-east-1",
    model_kwargs={
        "temperature": 0.5,
        "max_tokens": 2048,
        "top_p": 0.9,
    }
)

## Calling subagents

In [13]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage

@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content


main_agent = create_agent(
    model=llm,
    
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.")

## Test

In [14]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [15]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='4998b603-48e9-4f10-b45e-86994385bfec'),
              AIMessage(content=[{'type': 'text', 'text': '<thinking> The User has asked for the square root of 456. To find this, I will use the available tool that calculates the square root of a number. </thinking>\n'}, {'type': 'tool_use', 'name': 'call_subagent_1', 'input': {'x': 456}, 'id': 'tooluse_vvLN_TEvTl6ybQCpafNsog'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'eca2f6be-f891-4db6-82ca-e5e9b13fa170', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 02 Jan 2026 21:45:57 GMT', 'content-type': 'application/json', 'content-length': '466', 'connection': 'keep-alive', 'x-amzn-requestid': 'eca2f6be-f891-4db6-82ca-e5e9b13fa170'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [517]}, 'model_provider': 'bedrock_converse', 'model_name': 'amazon.nova-micro-v1:0'}, id='

In [16]:
from pprint import pformat
from IPython.display import Markdown, display

# Convierte el diccionario a string formateado y lo envuelve en un bloque de código
display(Markdown(f"```python\n{pformat(response)}\n```"))

```python
{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='4998b603-48e9-4f10-b45e-86994385bfec'),
              AIMessage(content=[{'type': 'text', 'text': '<thinking> The User has asked for the square root of 456. To find this, I will use the available tool that calculates the square root of a number. </thinking>\n'}, {'type': 'tool_use', 'name': 'call_subagent_1', 'input': {'x': 456}, 'id': 'tooluse_vvLN_TEvTl6ybQCpafNsog'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'eca2f6be-f891-4db6-82ca-e5e9b13fa170', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 02 Jan 2026 21:45:57 GMT', 'content-type': 'application/json', 'content-length': '466', 'connection': 'keep-alive', 'x-amzn-requestid': 'eca2f6be-f891-4db6-82ca-e5e9b13fa170'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [517]}, 'model_provider': 'bedrock_converse', 'model_name': 'amazon.nova-micro-v1:0'}, id='lc_run--019b80ac-ac20-71c1-a777-83e02f5be1a9-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 456}, 'id': 'tooluse_vvLN_TEvTl6ybQCpafNsog', 'type': 'tool_call'}], usage_metadata={'input_tokens': 485, 'output_tokens': 61, 'total_tokens': 546, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}),
              ToolMessage(content='The square root of 456.0 is approximately **21.35**.', name='call_subagent_1', id='be21f1f7-c542-406c-97fd-8c9ff9361b4b', tool_call_id='tooluse_vvLN_TEvTl6ybQCpafNsog'),
              AIMessage(content='The square root of 456 is approximately **21.35**.', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'ddaae453-258e-45bb-872d-5d16a67ee2c3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 02 Jan 2026 21:45:59 GMT', 'content-type': 'application/json', 'content-length': '254', 'connection': 'keep-alive', 'x-amzn-requestid': 'ddaae453-258e-45bb-872d-5d16a67ee2c3'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [264]}, 'model_provider': 'bedrock_converse', 'model_name': 'amazon.nova-micro-v1:0'}, id='lc_run--019b80ac-b664-76f2-a2f0-50004007ef90-0', usage_metadata={'input_tokens': 596, 'output_tokens': 18, 'total_tokens': 614, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})]}
```

In [17]:
from IPython.display import JSON

JSON(response)

<IPython.core.display.JSON object>

In [ ]:
import json
from IPython.display import Markdown, display

# El parámetro default=str es útil para manejar objetos que no son 
# serializables directamente a JSON (como las clases de LangChain)
json_str = json.dumps(response, indent=2, default=str)
display(Markdown(f"```json\n{json_str}\n```"))